# Requêtes APIs Orion

Bienvenue dans la documentation de ce notebook de test d'APIs permettant de récupérer des données utiles pour la création d'un score de risque pour les feux de forêt.


## Introduction

Les feux de forêt sont l'un des fléaux les plus dévastateurs pour les écosystèmes et les communautés qui dépendent d'eux. Dans de nombreux cas, la prévention est la meilleure solution, et cela nécessite des outils de prévision précis pour mesurer le risque de déclenchement de feux de forêt.

Ce notebook a pour objectif de tester différentes APIs pour créer un score de risque de feux de forêt en utilisant des informations telles que la météo, la végétation, l'humidité des sols, la topographie, et d'autres données environnementales. En explorant différentes sources de données, nous espérons trouver les meilleures informations pour créer un modèle de prédiction précis qui aidera les pompiers à prendre des mesures pour prévenir les feux de forêt.

Nous avons sélectionné quelques APIs mentionnées dans le papier [Wildfire Danger Prediction and Understanding With Deep Learning](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2022GL099368) et nous allons explorer chacune d'entre elles en détail. Cette documentation servira de guide pour comprendre comment requêter et utiliser ces APIs.

Nous espérons que ce notebook sera utile pour les personnes qui cherchent à améliorer la prévention des feux de forêt et à protéger nos écosystèmes.


### Comment commencer ?
#### Clonage du repo
Commencer par ouvrir son IDE (Pycharm, VS, terminal...), et cloner le repository Git : 
```
git clone git@github.com:pyronear/pyro-risks.git
```

/!\ Il faut que votre clé SSH soit configurée dans Github. Si ce n'est pas fait, voir [ici](https://docs.github.com/en/authentication/connecting-to-github-with-ssh/adding-a-new-ssh-key-to-your-github-account).

#### Environnement virtuel
Activer son [environnement virtuel](https://realpython.com/python-virtual-environments-a-primer/) :
```bash
source venv/bin/activate
```

/!\ Si vous n'avez pas configuré votre environnement virtuel, rendez vous à la racine du repo, et tapez dans votre terminal : 
```bash
python -m venv venv
pip install -r requirements.txt
```

#### Notebook
Lancez ensuite votre notebook :

```bash
pip install jupyter
jupyter notebook
```

Une fenêtre s'ouvre dans votre navigateur, c'est le notebook !
 
Se rendre dans `pyro_risks/notebooks`, puis vous pouvez créer votre propre notebook, ou bien lancer un notebook déjà créé.

## Requête des APIs mentionnées dans le papier

Le papier mentionne 6 datasets : 

 - **Daily weather data**, de **[ERA-5 Land](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview)** (Muñoz-Sabater et al., 2021) (température maximale à 2 m, vitesse maximale du vent, humidité relative minimale, précipitations totales, température maximale du point de rosée à 2 m et la pression maximale en surface), aggrégées à la journée.

 - **Satellite variables** de **[MODIS](https://www.earthdata.nasa.gov/learn/find-data/near-real-time/firms/mcd14dl-nrt)**, dont Normalized Difference Vegetation Index (NDVI; Didan, 2015), day and night Land Surface Temperature (LST; Wan et al., 2015).

 - **Soil moisture index** de **[European Drought Observatory](https://edo.jrc.ec.europa.eu/edov2/php/index.php?id=1000)** (Cammalleri et al., 2017).

 - **Roads distance, waterway distance, and yearly population density** de **WorldPop** (Tatem, 2017).

 - **Elevation and Slope** de **[Copernicus EU-DEM](https://land.copernicus.eu/imagery-in-situ/eu-dem/eu-dem-v1.1)** (Bashfield & Keim, 2011).

 - **Ten variables with the fraction of classes** (?) de **[Copernicus Corine Land Cover](https://land.copernicus.eu/pan-european/corine-land-cover)** (Büttner, 2014).


### 1. Daily weather data - ERA-5 Land

Documentation [ici](https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation).

In [8]:
import cdsapi
c = cdsapi.Client(url=cfg.CDS_URL, key=f"{cfg.CDS_UID}:{cfg.CDS_API_KEY}", verify=0)
c.retrieve(
    'reanalysis-era5-land',
    {
        'format':'grib',
        'time':[
            '00:00','01:00','02:00',
            '03:00','04:00','05:00',
            '06:00','07:00','08:00',
            '09:00','10:00','11:00',
            '12:00','13:00','14:00',
            '15:00','16:00','17:00',
            '18:00','19:00','20:00',
            '21:00','22:00','23:00'
        ],
        'day':'01',
        'month':'01',
        'year':'2013',
        'variable':'snow_depth_water_equivalent',
    },
    'snow_20130101_era5land.grib')

NameError: name 'cfg' is not defined

### 2. Satellite variables - MODIS

### 3. Soil moisture index - European Drought Observatory

### 4. Roads distance, waterway distance, and yearly population density - WorldPop

### 5. Elevation and Slope - Copernicus EU-DEM

### 6. Ten variables with the fraction of classes - Copernicus Corine Land Cover